In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential, losses, metrics, activations
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def normalize(x):
  # Normalizes to [-1, 1]
  return (x - 127.5) / 127.5

In [ ]:
# Loads data for MNIST

BATCH_SIZE = 64

(x_train, y_train), (x_validation, y_validation) = mnist.load_data()

x_validation, y_validation = x_validation[:500], y_validation[:500]

x_train, x_validation = normalize(x_train), normalize(x_validation)

x_train, x_validation = np.reshape(x_train, (x_train.shape[0], -1)), np.reshape(x_validation, (x_validation.shape[0], -1))

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
         .shuffle(buffer_size = 50_000) \
         .batch(BATCH_SIZE, drop_remainder=True)

# Klassifisering

In [ ]:
# Oppvarming
# Definer en classifier-modell for MNIST. Denne arkitekturen kan du enkelt gjøre
# om til discriminator-modell senere ved å endre antall outputs fra 10 til 1.
#
# Bruk et enkelt nevralt nettverk her (ikke CNNs). 
#
# Sikt gjerne på minst 97 % validation accuracy.

classifier = None

classifier.compile(metrics = [metrics.SparseCategoricalAccuracy()])


In [ ]:
# Her hadde det desidert enkleste vært å trene classifier-modellen med 
# classifier.fit(). I stedet demonstreres hvordan man kan trene
# modellen ved hjelp av det funksjonelle API-et til Tensorflow. Dette API-et
# blir nyttig med en gang man skal ha et mer kompliserte treningssteg, slik som 
# for GAN. 
#
# Her brukes en GradientTape for å overvåke utregningen som inngår under tape-
# uttrykket (forward pass gjennom modellen). Deretter kan man hente ut 
# gradientene for en utregning Y med hensyn til variablene W, og bruke 
# gradientene til å oppdatere modellen som vanlig (med apply_gradients). 

classification_optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step_classifier(batch_x, batch_y):
  with tf.GradientTape() as tape:

    output = classifier(batch_x)

    loss = 0 # TODO 

  gradients = tape.gradient(loss, classifier.trainable_variables)
  classification_optimizer.apply_gradients(zip(gradients, classifier.trainable_variables))

  return loss,  


validation_accuracy = []
for epoch in range(10):
  
  print("Epoch", epoch)
  for (batch_x, batch_y) in dataset:
    train_step_classifier(batch_x, batch_y)
  
  accuracy = classifier.evaluate(x_validation, y_validation)[1]
  validation_accuracy.append(accuracy)

plt.plot(validation_accuracy)

# General Adverserial Networks

In [ ]:
# Definer diskriminator-modellen. I motsetning til klassifikatoren skal denne gi
# ut én output-verdi: hvor vidt bilde-input-en er autentisk (true) eller 
# generert (false). 

discriminator = None

discriminator.compile()
discriminator.summary()

In [ ]:
noise_dim = 20

# Definer generator-modellen. Denne skal ta inn en støy-vektor (en vektor med 
# størrelse lik noise_dim av tilfeldige flyttall) og gi ut et bilde. For MNIST
# må den dermed ha 28*28=784 outputs. 
#
# Du velger størrelsen på noise_dim selv. 

generator = None

generator.compile()
generator.summary()

In [ ]:
def to_image(img):
  return img.reshape([28, 28])

def show_image(img):
  img = to_image(img)
  plt.imshow(img, cmap='gray')
  plt.show()

def generate_noise(dim, batch_size = 1):
  return tf.random.normal([batch_size, dim])

# Generer og viser et bilde fra den utrente generatoren

generated_image = generator(generate_noise(noise_dim)).numpy()

show_image(generated_image)


In [ ]:
# Implementer treningssteget. Denne tar inn én batch med treningsbilder.
# Denne skal generere et sett med falske bilder. Diskriminatoren skal vurdere
# både de falske og de ekte bildesettene. Begge modellene skal oppdateres
# ut fra vurderingen til diskriminatoren.
#
# Tips: bruk gjerne en Optimizer for hver av generator-modellen og diskriminator-
# modellen.
#
# Returnerer: 
# D_loss: loss-verdien for diskriminatoren
# G_loss: loss-verdien for generatoren
# P_real: Diskriminatoren sin accuracy for ekte bilder
# P_fake: Diskriminatoren sin accuracy for falske bilder

@tf.function
def train_step(real_images):
  
  D_loss = None
  G_loss = None
  P_real = None
  P_fake = None

  return D_loss, G_loss, P_real, P_fake


In [ ]:
def show_image_grid(images):
  N = images.shape[0]

  fig = plt.figure(figsize=(3, 3))
  fig.set_size_inches(10, 10)

  for i in range(N):
      plt.subplot(3, 3, i+1)
      img = to_image(images[i])
      plt.imshow(img, cmap="gray")
      plt.axis('off')

  plt.show()

In [ ]:
EPOCHS = 100

noise_for_training_visualization = generate_noise(noise_dim, batch_size = 9)

for epoch in range(EPOCHS):

  print("Epoch", epoch),

  for step, (x_train, _) in enumerate(dataset):
    Ld, Lg, Pr, Pf = train_step(x_train)

    if step % 100 == 0:
      print("Step {}. Ld={}, Lg=={}, Pr={}, Pf={}".format(step, Ld, Lg, Pr, Pf))

  show_image_grid(generator(noise_for_training_visualization).numpy())
  

In [ ]:
# Denne kan du bruke til å utforske generatoren!

from ipywidgets import interact

p1 = generate_noise(noise_dim)
p2 = generate_noise(noise_dim)
p3 = generate_noise(noise_dim)

@interact(k1=(0, 1.0), k2=(0, 1.0))
def g(k1, k2):
    p = p1 + k1 * (p2 - p1) + k2 * (p3 - p1)
    show_image(generator(p).numpy())